This notebook is for the testing of the thread reconstruction hypothesys.

In [ ]:
import json
from sklearn.metrics import f1_score
from transformers import AutoTokenizer
from transformers import BertForNextSentencePrediction
from transformers import Trainer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from src.thread_reconstruction import get_dummy, get_edge_matrix_from_annot, greedify_binary, g_binary, g_probas, estimate_replies, binarize, greedify_probas

In [ ]:
with open("project-1-at-2023-03-08-06-54-9ca75886.json") as f:
    annot1 = json.load(f)
    
with open("project-1-at-2023-03-14-18-09-51c2cb5b.json") as f:
    annot2 = json.load(f)

annot = annot1 + annot2

In [ ]:
model = BertForNextSentencePrediction.from_pretrained('models/trained_models/results_fine_tuned_ru_conversational_bert/checkpoint-150000/')
tokenizer = AutoTokenizer.from_pretrained("models/ru_conversational_cased_L-12_H-768_A-12_pt_v1")
trainer = Trainer(model=model)

In [ ]:
models = [
    ("dummy", get_dummy),
    ("greedify_binary", g_binary),
    ("greedigy_probas", g_probas)
]

In [ ]:
report = {
    "model_name": [],
    "precision": [],
    "recall": [],
    "f1":[]
         }
for a in annot:
    for offset in [3,7,15]:
        dialog = a['annotations'][0]["result"]
        gold_matrix = get_edge_matrix_from_annot(dialog, 100)
        dial = [x["text"].split("|")[-1].strip() for x in a['data']["dialogue"]]
        estimanted_matrix_proba = estimate_replies(dial, offset , trainer)
        for name, model in models:
            if "dummy" != name:
                name = f"{name}_{offset}"
            report["model_name"].append(name)
            report["precision"].append(precision_score(gold_matrix.flatten(), model(estimanted_matrix_proba).flatten(),average="macro"))
            report["recall"].append(recall_score(gold_matrix.flatten(), model(estimanted_matrix_proba).flatten(),average="macro"))
            report["f1"].append(f1_score(gold_matrix.flatten(), model(estimanted_matrix_proba).flatten(), average="macro"))

In [ ]:
#dialog_text = [x["text"].split("|")[-1].strip() for x in a['data']["dialogue"]]
#dialog_text = np.array(dialog_text)
#print(dialog_text[[66, 76]])

In [ ]:
df = pd.DataFrame(report)
print(df.groupby("model_name").agg("mean").to_latex())

In [ ]:
print(classification_report(gold_matrix.flatten(),
                            greedify_probas(estimanted_matrix_proba).flatten()))

In [ ]:
print(classification_report(gold_matrix.flatten(),
                            greedify_binary(binarize(estimanted_matrix_proba)).flatten()))

In [ ]:
print(classification_report(gold_matrix.flatten(),
                            binarize(estimanted_matrix_proba).flatten()))